In [1]:
import numpy as np
import pandas as pd

In [2]:
train_cust =pd.read_csv('train_customers.csv',parse_dates=['created_at','updated_at'])
test_cust =pd.read_csv('test_customers.csv',parse_dates=['created_at','updated_at'])
train_loc =pd.read_csv('train_locations.csv')
test_loc =pd.read_csv('test_locations.csv')
vendors =pd.read_csv('vendors.csv',parse_dates=['created_at','updated_at'])
orders =pd.read_csv('orders.csv',parse_dates=['created_at','delivery_date'])
submission =pd.read_csv('SampleSubmission (1).csv')

C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_cust= train_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)
test_cust= test_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)

In [4]:
train_loc.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823


In [5]:
train_cust.head()

,customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [6]:
test_cust=test_cust[['customer_id','created_at','updated_at']]
train_cust=train_cust[['customer_id','created_at','updated_at']];train_cust.head()

,customer_id,created_at,updated_at
0,TCHWPBT,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,2018-03-15 19:57:01,2018-03-15 19:57:01


In [7]:
print(train_cust.shape,test_cust.shape,submission.shape, vendors.shape)

(34674, 3) (9768, 3) (1672000, 2) (100, 59)


### DROP DUPLICATES  CUSTOMERS

In [8]:
test_cust = test_cust.drop_duplicates(subset='customer_id', keep="first")
train_cust = train_cust.drop_duplicates(subset='customer_id', keep="first")

In [9]:
test_cust.shape

(9753, 3)

### MERGE

In [10]:
train =pd.merge(train_cust, train_loc, on ='customer_id', how ='left')
test =pd.merge(test_cust, test_loc, on ='customer_id', how ='left')

In [11]:
test.head(2)

,customer_id,created_at,updated_at,location_number,location_type,latitude,longitude
0,ICE2DJP,2018-02-07 16:45:36,2018-02-07 16:45:36,0,NaN,-96.407538,-67.197291
1,ICE2DJP,2018-02-07 16:45:36,2018-02-07 16:45:36,1,NaN,0.038654,-78.595477


In [12]:
test.shape

(16317, 7)

In [13]:
vendors.head(2)

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1


In [14]:
test=test.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)
train=train.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

In [15]:
print(train.shape, test.shape, submission.shape)

(5802400, 66) (1631700, 66) (1672000, 2)


# Finding common rows (intersection) in two Pandas dataframes

In [16]:
submission.columns

Index(['CID X LOC_NUM X VENDOR', 'target'], dtype='object')

In [17]:
test['location_number_obj']=test['location_number'].astype(str)
test['id_obj']=test['id'].astype(str)
train['location_number_obj']=train['location_number'].astype(str)
train['id_obj']=train['id'].astype(str)

In [18]:
test['CID X LOC_NUM X VENDOR'] = test['customer_id'] +' X '+ test['location_number_obj'] +' X '+ test['id_obj']
train['CID X LOC_NUM X VENDOR'] = train['customer_id'] +' X '+ train['location_number_obj'] +' X '+ train['id_obj']

In [19]:
test = pd.merge(test, submission, how='outer', on=['CID X LOC_NUM X VENDOR'])

In [20]:
print(test.shape, submission.shape, train.shape)

(1672000, 70) (1672000, 2) (5802400, 69)


In [21]:
all_services = orders['CID X LOC_NUM X VENDOR'].values.tolist()
train['target']= train['CID X LOC_NUM X VENDOR'].isin(all_services).astype(int).values

In [22]:
train['target'].value_counts()

0    5776323
1      26077
Name: target, dtype: int64

In [23]:
test.drop('target', axis=1, inplace =True)

In [24]:
print(test.shape,train.shape)

(1672000, 69) (5802400, 70)


In [25]:
## Reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [26]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 2318.58 Mb (25.7% reduction)
Mem. usage decreased to 676.09 Mb (24.3% reduction)


In [ ]:
train.to_csv('akeelah_train.csv', index =False)
test.to_csv('akeelah_test.csv', index =False)